In [3]:
from bart_survival import surv_bart as sb

WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [7]:
from bart_survival import simulation
import numpy as np

In [8]:
rng = np.random.default_rng(1)
x_mat = simulation.get_x_matrix(
    N=100,
    x_vars=1,
    VAR_CLASS=[2],
    VAR_PROB=[.5],
    rng = rng
)
event_dict, sv_true, sv_scale_true = simulation.simulate_survival(
    x_mat = x_mat,
    scale_f = "np.exp(2 + .4*x_mat[:,0])",
    shape_f = "1",
    eos = 20,
    cens_scale=None,
    time_scale=5,
    true_only=False,
    rng = rng
)
# prepare data
t_scale = sb.get_time_transform(event_dict["t_event"], time_scale = 5)
y_sk = sb.get_y_sklearn(event_dict["status"], t_scale)
trn = sb.get_surv_pre_train(y_sk, x_mat, weight=None)
post_test = sb.get_posterior_test(y_sk=y_sk, x_test = x_mat)

mean shape 1.0
mean scale 9.315139848237454
mean time draws 8.513352463409824


In [18]:

SPLIT_RULES =  [
    "pmb.ContinuousSplitRule()", # time
    "pmb.OneHotSplitRule", # ccsr_ind_p2
]
model_dict = {"trees": 50,
    "split_rules": SPLIT_RULES
}
sampler_dict = {
            "draws": 10,
            "tune": 10,
            "cores": 2,
            "chains": 2,
            "compute_convergence_checks": False
        }

BSM = sb.BartSurvModel(model_config=model_dict, sampler_config=sampler_dict)

BSM.fit(
    y =  trn["y"],
    X = trn["x"],
    weights=trn["w"],
    coords = trn["coord"],
    random_seed=5
) 

Only 10 samples in chain.
Multiprocess sampling (2 chains in 2 jobs)
PGBART: [f]


Sampling 2 chains for 10 tune and 10 draw iterations (20 + 20 draws total) took 5 seconds.


Inference data with groups:
	> posterior
	> sample_stats
	> observed_data
	> constant_data
	> fit_data
	> predictor_names
	> offset

In [19]:
import multiprocessing as mp 

childs = mp.active_children()

In [20]:
childs

[<ForkProcess name='SyncManager-26' pid=32592 parent=30054 started>,
 <ForkProcess name='SyncManager-10' pid=30961 parent=30054 started>,
 <ForkProcess name='SyncManager-19' pid=32410 parent=30054 started>,
 <ForkProcess name='SyncManager-1' pid=30302 parent=30054 started>]